# ENTREGABLE 1

INSTRUCCIONES

Realizar la primera fase del análisis exploratorio (limpieza) usando el archivo CSV (dataset_banco.csv) con 45215 filas y 17 columnas.

# 1. El problema del negocio

Una entidad bancaria contrata a una empresa de marketing encargada de contactar telefónicamente a posibles clientes para determinar si están interesados o no en adquirir un certificado de depósito a término con el banco.

¿Qué perfil tienen los clientes con mayor potencial de conversión?

# 2. El set de datos

Cada registro contiene 16 características (las primeras 16 columnas) y una categoría ("yes" o "no" dependiendo de si la persona está o no interesada en adquirir el producto). Las columnas son:

1. "age":  edad (numérica)
2. "job": tipo de trabajo (categórica: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar","self-employed", "retired", "technician", "services")
3. "marital": estado civil (categórica: "married", "divorced", "single")
4. "education": nivel educativo (categórica: "unknown", "secondary", "primary", "tertiary")
5. "default": si dejó de pagar sus obligaciones (categórica: "yes", "no")
6. "balance": saldo promedio anual en euros (numérica)
7. "housing": ¿tiene o no crédito hipotecario? (categórica: "yes", "no")
8. "loan": ¿tiene créditos de consumo? (categórica: "yes", "no")
9. "contact": medio a través del cual fue contactado (categórica: "unknown", "telephone", "cellular")
10. "day": último día del mes en el que fue contactada (numérica)
11. "month": último mes en el que fue contactada (categórica: "jan", "feb", "mar", ..., "nov", "dec")
12. "duration": duración (en segundos) del último contacto (numérica)
13. "campaign": número total de veces que fue contactada durante la campaña (numérica)
14. "pdays": número de días transcurridos después de haber sido contactado antes de la campaña actual (numérica. -1 indica que no fue contactado previamente)
15. "previous": número de veces que ha sido contactada antes de esta campaña (numérica)
16. "poutcome": resultado de la campaña de marketing anterior (categórica: "unknown", "other", "failure", "success")
17. "y": categoría ¿el cliente se suscribió a un depósito a término? (categórica: "yes", "no")

# 3. Una primera mirada al dataset

In [115]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [116]:
# Lectura
df = pd.read_csv("dataset_banco.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45215 entries, 0 to 45214
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        45215 non-null  int64  
 1   job        45213 non-null  object 
 2   marital    45214 non-null  object 
 3   education  45214 non-null  object 
 4   default    45215 non-null  object 
 5   balance    45213 non-null  float64
 6   housing    45215 non-null  object 
 7   loan       45215 non-null  object 
 8   contact    45215 non-null  object 
 9   day        45215 non-null  int64  
 10  month      45215 non-null  object 
 11  duration   45214 non-null  float64
 12  campaign   45215 non-null  int64  
 13  pdays      45214 non-null  float64
 14  previous   45215 non-null  int64  
 15  poutcome   45215 non-null  object 
 16  y          45215 non-null  object 
dtypes: float64(3), int64(4), object(10)
memory usage: 5.9+ MB


In [117]:
print(df.shape)
df.head()

(45215, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


In [118]:
# Veamos las variables categóricas y las numéricas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45215 entries, 0 to 45214
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        45215 non-null  int64  
 1   job        45213 non-null  object 
 2   marital    45214 non-null  object 
 3   education  45214 non-null  object 
 4   default    45215 non-null  object 
 5   balance    45213 non-null  float64
 6   housing    45215 non-null  object 
 7   loan       45215 non-null  object 
 8   contact    45215 non-null  object 
 9   day        45215 non-null  int64  
 10  month      45215 non-null  object 
 11  duration   45214 non-null  float64
 12  campaign   45215 non-null  int64  
 13  pdays      45214 non-null  float64
 14  previous   45215 non-null  int64  
 15  poutcome   45215 non-null  object 
 16  y          45215 non-null  object 
dtypes: float64(3), int64(4), object(10)
memory usage: 5.9+ MB


# 4. Limpieza

Realizaremos el proceso de limpieza teniendo en cuenta las situaciones más comunes:

1. Datos faltantes en algunas celdas
2. Columnas irrelevantes (que no responden al problema que queremos resolver)
3. Registros (filas) repetidos
4. Valores extremos (*outliers*) en el caso de las variables numéricas. Se deben analizar en detalle pues no necesariamente la solución es eliminarlos
5. Errores tipográficos en el caso de las variables categóricas

Al final de este proceso de limpieza deberíamos tener un set de datos **íntegro**, listo para la fase de Análisis Exploratorio.

## 4.1 Datos faltantes

Comenzamos a ver que los datos no están completos, pues no todas las columnas tienen la misma cantidad de registros.

El número total de registros debería ser 45.215. Sin embargo columnas como "job", "marital", "education", "balance", "duration" y "pdays".

Por ser tan pocos los datos  faltantes optaremos por eliminar las filas correspondientes:

In [119]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45207 entries, 0 to 45214
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        45207 non-null  int64  
 1   job        45207 non-null  object 
 2   marital    45207 non-null  object 
 3   education  45207 non-null  object 
 4   default    45207 non-null  object 
 5   balance    45207 non-null  float64
 6   housing    45207 non-null  object 
 7   loan       45207 non-null  object 
 8   contact    45207 non-null  object 
 9   day        45207 non-null  int64  
 10  month      45207 non-null  object 
 11  duration   45207 non-null  float64
 12  campaign   45207 non-null  int64  
 13  pdays      45207 non-null  float64
 14  previous   45207 non-null  int64  
 15  poutcome   45207 non-null  object 
 16  y          45207 non-null  object 
dtypes: float64(3), int64(4), object(10)
memory usage: 6.2+ MB


## 4.2 Columnas irrelevantes

Una columna irrelevante puede ser:

- **Una columna que no contiene información relevante para el problema que queremos resolver**. Por ejemplo en este caso podría ser una columna que no guarde relación con el posible perfil del cliente (deporte favorito, hobbies, comida favorita, etc.)
- **Una columna categórica pero con un sólo nivel**. Por ejemplo si en la columna "job" solo tuviésemos el nivel "unknown".
- **Una columna numérica pero con un sólo valor**. Por ejemplo si en la columna "edad" todos los vlaores fuesen iguales a 50.
- **Columnas con información redundante**. Por ejemplo si además de las columnas "month" y "day" tuviésemos la columna "month-day", resultado de combinar las dos anteriores.

Si tenemos la duda de si alguna columna puede ser relevante o no lo mejor es dejarla (y más adelante en posteriores etapas podremos darnos cuenta de si se debe preservar o no).

En este caso todas las columnas pueden resultar relevantes, pero debemos verificar que no haya columnas categóricas con un sólo nivel, o columnas numéricas con un sólo valor:

### Age

In [120]:
df['age'].describe()

count    45207.000000
mean        41.005596
std         12.037399
min         18.000000
25%         33.000000
50%         39.000000
75%         48.000000
max        776.000000
Name: age, dtype: float64

### Job

In [121]:
print(df['job'].value_counts().sort_index())

job
MANAGEMENT           2
Management           2
Retired              1
Self-employed        1
Services             1
admin.            5166
administrative       3
blue-collar       9730
entrepreneur      1487
housemaid         1240
management        9454
retired           2263
self-employed     1578
services          4153
student            938
technician        7597
unemployed        1303
unknown            288
Name: count, dtype: int64


### Marital

In [122]:
print(df['marital'].value_counts().sort_index())

marital
DIVORCED        3
Single          4
div.            7
divorced     5196
married     27211
single      12786
Name: count, dtype: int64


### Education

In [123]:
print(df['education'].value_counts().sort_index())

education
Primary          2
SECONDARY        3
Secondary        1
Tertiary         1
UNK              2
primary       6848
sec.             2
secondary    23192
tertiary     13301
unknown       1855
Name: count, dtype: int64


### Default

In [124]:
print(df['default'].value_counts().sort_index())

default
no     44391
yes      816
Name: count, dtype: int64


### Balance

In [125]:
df['balance'].describe()

count     45207.000000
mean       1374.201318
std        3924.491665
min       -8019.000000
25%          72.000000
50%         448.000000
75%        1427.500000
max      527532.000000
Name: balance, dtype: float64

### Housing

In [126]:
print(df['housing'].value_counts().sort_index())

housing
no     20082
yes    25125
Name: count, dtype: int64


### Loan

In [127]:
print(df['loan'].value_counts().sort_index())

loan
NO         2
No         5
YES        2
Yes        1
no     37956
yes     7241
Name: count, dtype: int64


### Contact

In [128]:
print(df['contact'].value_counts().sort_index())

contact
cellular     29278
mobile           3
phone            3
telephone     2902
unknown      13021
Name: count, dtype: int64


### Day

In [129]:
df['day'].describe()

count    45207.000000
mean        15.806534
std          8.323015
min          1.000000
25%          8.000000
50%         16.000000
75%         21.000000
max         31.000000
Name: day, dtype: float64

### Month

In [130]:
print(df['month'].value_counts().sort_index())

month
apr     2932
aug     6247
dec      214
feb     2649
jan     1403
jul     6895
jun     5342
mar      477
may    13760
nov     3971
oct      738
sep      579
Name: count, dtype: int64


### Duration

In [131]:
df['duration'].describe()

count    45207.000000
mean       258.032539
std        257.460759
min      -1389.000000
25%        103.000000
50%        180.000000
75%        319.000000
max       4918.000000
Name: duration, dtype: float64

### Campaign

In [132]:
df['campaign'].describe()

count    45207.000000
mean         2.763731
std          3.098058
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         63.000000
Name: campaign, dtype: float64

### Pdays

In [133]:
df['pdays'].describe()

count    45207.000000
mean        40.178225
std        100.103283
min         -1.000000
25%         -1.000000
50%         -1.000000
75%         -1.000000
max        871.000000
Name: pdays, dtype: float64

### Previous

In [134]:
df['previous'].describe()

count    45207.000000
mean         0.580198
std          2.303341
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        275.000000
Name: previous, dtype: float64

### Poutcome

In [135]:
print(df['poutcome'].value_counts().sort_index())

poutcome
Success        2
UNK            4
failure     4900
other       1838
success     1509
unknown    36954
Name: count, dtype: int64


### Y

In [136]:
print(df['y'].value_counts().sort_index())

y
no     39919
yes     5288
Name: count, dtype: int64


Todas las columnas categóricas tienen más de 1 subnivel. No eliminaremos ninguna.

Verifiquemos lo que ocurre con las columnas numéricas:

Todas las columnas numéricas tienen desviaciones estándar ("std") diferentes de cero, lo que indica que no tienen un único valor.

Preservaremos todas las columnas numéricas.

##4.3 Filas repetidas

In [137]:
df.drop_duplicates(inplace=True)

##4.4 *Outliers* en las variables numéricas

No siempre se deben eliminar los *outliers* porque dependiendo de la variable numérica analizada estos pueden contener información importante.


In [138]:
age_eliminar = df[df['age'] > 100].index
df = df.drop(age_eliminar)
df['age'].describe()

count    45195.000000
mean        40.936608
std         10.618241
min         18.000000
25%         33.000000
50%         39.000000
75%         48.000000
max         95.000000
Name: age, dtype: float64

In [139]:
eliminar_duration = df[df['duration'] <= 0].index
df = df.drop(eliminar_duration)
df['duration'].describe()

count    45190.000000
mean       258.117017
std        257.347108
min          1.000000
25%        103.000000
50%        180.000000
75%        319.000000
max       4918.000000
Name: duration, dtype: float64

In [140]:
eliminar_previous = df[df['previous'] >  100].index
df = df.drop(eliminar_previous)
df['previous'].describe()

count    45189.000000
mean         0.574166
std          1.907950
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         58.000000
Name: previous, dtype: float64

##4.5 Errores tipográficos en variables categóricas

En una variable categórica pueden aparecer sub-niveles como "unknown" y "UNK" que para nosotros son equivalentes pero que para nuestro programa parecerían diferentes.

Se deben unificar estos sub-niveles

In [141]:
df['job'] = df['job'].str.lower()
df['job'] = df['job'].replace('admin.','administrative')
print(df['job'].value_counts())

job
blue-collar       9727
management        9455
technician        7592
administrative    5168
services          4152
retired           2263
self-employed     1578
entrepreneur      1486
unemployed        1303
housemaid         1240
student            937
unknown            288
Name: count, dtype: int64


In [142]:
df['marital'] = df['marital'].str.lower()
df['marital'] = df['marital'].replace('div.','divorced')
print(df['marital'].value_counts())

marital
married     27200
single      12785
divorced     5204
Name: count, dtype: int64


In [143]:
df['education'] = df['education'].str.lower()
df['education'] = df['education'].replace('sec.','secondary')
df['education'] = df['education'].replace('unk','unknown')
print(df['education'].value_counts())

education
secondary    23187
tertiary     13298
primary       6847
unknown       1857
Name: count, dtype: int64


In [144]:
df['contact'] = df['contact'].replace('phone','telephone')
df['contact'] = df['contact'].replace('mobile','cellular')
print(df['contact'].value_counts())

contact
cellular     29274
unknown      13011
telephone     2904
Name: count, dtype: int64


In [145]:
df['poutcome'] = df['poutcome'].str.lower()
df['poutcome'] = df['poutcome'].replace('unk','unknown')
print(df['poutcome'].value_counts())

poutcome
unknown    36943
failure     4898
other       1837
success     1511
Name: count, dtype: int64


## Conclusiones

In [153]:
print(df.shape)
df.head()

(45189, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


In [147]:
# df.to_csv(dataset_banco_clean.csv, index=False)